# Built-in Modules

- [Item 42: Define Function Decorators with *functools.wraps*](http://localhost:8888/notebooks/Built-in-Modules.ipynb#Item-42:-Define-Function-Decorators-with-functools.wraps)
- [Item 43: Consider contextlib and with Statements for Resuable try/finally Behavior](#Item-43:-Consider-contextlib-and-with-Statements-for-Resuable-try/finally-Behavior)

## Item 42: Define Function Decorators with *functools.wraps*

In [ ]:
def trace(func):
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

fibonacci(3)

print(fibonacci)
help(fibonacci)

from functools import wraps

def trace(func):
    @wraps(func)
    def wrapper(*args, **kwargs):
        result = func(*args, **kwargs)
        print('%s(%r, %r) -> %r' %
              (func.__name__, args, kwargs, result))
        return result
    return wrapper

@trace
def fibonacci(n):
    """Return the n-th Fibonacci number"""
    if n in (0, 1):
        return n
    return (fibonacci(n - 2) + fibonacci(n - 1))

# fibonacci = trace(fibonacci)

print(fibonacci)
help(fibonacci)

### Things to Remember

- Decorators are Python syntax for allowing one function to modify another function at runtime.
- Using decorators can cause strange behaviors in tools that do introspection, such as debuggers.
- Use the **wraps** decorator from the **functools** built-in module when you define your own decorators to avoid any issues.

## Item 43: Consider *contextlib* and *with* Statements for Resuable *try/finally* Behavior

In [ ]:
from threading import Lock

lock = Lock()
with lock:
    print('Lock is held')
    
# equivalent
lock.acquire()
try:
    print('Lock is held')
finally:
    lock.release()

The **with** statement version of this is better because it eliminates the need to write the repetitive code of the **try/finally** construction.

In [ ]:
import logging

def my_function():
    logging.debug('Some debug data')
    logging.error('Error log here')
    logging.debug('More debug data')
    
my_function()

from contextlib import contextmanager

@contextmanager
def debug_logging(level):
    logger = logging.getLogger()
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield
    finally:
        logger.setLevel(old_level)
        
"""
The yield expression is the point at which 
the with block's contents will execute.
Any exceptions that happen in the with block
with be re-raised by the yield expression for
you to catch in the helper function
"""

with debug_logging(logging.DEBUG):
    logging.info('Inside')
    my_function()
logging.warning('After')
my_function()

In [ ]:
from contextlib import contextmanager

@contextmanager
def log_level(level, name):
    logger = logging.getLogger(name)
    old_level = logger.getEffectiveLevel()
    logger.setLevel(level)
    try:
        yield logger
    finally:
        logger.setLevel(old_level)
        
with log_level(logging.DEBUG, 'my-log') as logger:
    logger.debug('This is my message')
    logging.debug('This will not print')

logger = logging.getLogger('my-log')
logger.debug('Debug will not print')
logger.error('Error will print')

### Things to Remember

- The **with** statement allows you to reuse logic from **try/finally** blocks and reduce visual noise.
- The **contextlib** built-in module provides a **contextmanager** decorator that makes it easy to use your own functions in **with** statements.
- The value yielded by context managers is supplied to the as part of the **with** statement. It's useful for letting your code directly access the cause of the special context.